In [1]:
! pip install -U tokenizers

     |████████████████████████████████| 3.0 MB 3.5 MB/s eta 0:00:01
ERROR: transformers 2.11.0 has requirement tokenizers==0.7.0, but you'll have tokenizers 0.8.0 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 18 kB/s s eta 0:00:01    |███████▏                        | 92.1 MB 45.2 MB/s eta 0:00:08
     |████████████████████████████████| 50 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 42.3 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 49.1 MB/s eta 0:00:01
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=7005ccfacec2c50a000be4e44ed2ffec47452996212fd23eb8bceeb4f066a8c7
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: keras 2.4.0 has requirement tensorflow>=2.2.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attemp

In [3]:
import tokenizers

In [4]:
bwpt = tokenizers.BertWordPieceTokenizer(
    vocab_file=None,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)

In [5]:
bwpt.train(
    files=["../input/hindi-oscar-corpus/hi_dedup_1000.txt"],
    vocab_size=30000,
    min_frequency=3,
    limit_alphabet=1000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

In [6]:
bwpt.save_model("/kaggle/working/", "hindi")

['/kaggle/working/hindi-vocab.txt']

In [24]:
cd ../input/bertsrc/

/kaggle/input/bertsrc


In [8]:
!ls

CONTRIBUTING.md		    optimization_test.py
LICENSE			    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
README.md		    requirements.txt
__init__.py		    run_classifier.py
create_pretraining_data.py  run_classifier_with_tfhub.py
extract_features.py	    run_pretraining.py
modeling.py		    run_squad.py
modeling_test.py	    sample_text.txt
multilingual.md		    tokenization.py
optimization.py		    tokenization_test.py


In [9]:
!python create_pretraining_data.py \
    --input_file=/kaggle/input/hindi-oscar-corpus/hi_dedup_1000.txt \
    --output_file=/kaggle/working/tf_examples.tfrecord \
    --vocab_file=/kaggle/working/hindi-vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5

W0701 23:24:19.907574 140630622037824 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0701 23:24:19.907903 140630622037824 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0701 23:24:19.908287 140630622037824 module_wrapper.py:139] From /kaggle/input/bertsrc/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0701 23:24:19.943599 140630622037824 module_wrapper.py:139] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0701 23:24:19.945683 140630622037824 module_wrapper.py:139] From create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0701 23:24:19.945983 140630622037824 create_pretraining_data.py:4

I0701 23:24:25.747543 140630622037824 create_pretraining_data.py:166] Wrote 2561 total instances


In [10]:
!python run_pretraining.py \
    --input_file=gs://tf-lang-model_danzter/*.tfrecord \
    --output_dir=gs://tf-lang-model_danzter/model/ \
    --do_train=True \
    --do_eval=True \
    --bert_config_file=/kaggle/input/bert-base-uncased/config.json \
    --train_batch_size=32 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=20 \
    --num_warmup_steps=10 \
    --learning_rate=2e-5 \
    --use_tpu=True \
    --tpu_name=$TPU_NAME

W0701 23:24:30.052810 140205107017536 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0701 23:24:30.053118 140205107017536 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0701 23:24:30.053412 140205107017536 module_wrapper.py:139] From /kaggle/input/bertsrc/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0701 23:24:30.055700 140205107017536 module_wrapper.py:139] From run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

2020-07-01 23:24:30.056107: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file."

2020-07-01 23:24:33.283509: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Cancelled: GCE check skipped due to presence of $NO_GCE_CHECK environment variable.".
2020-07-01 23:24:33.302474: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Cancelled: GCE check skipped due to presence of $NO_GCE_CHECK environment variable.".
W0701 23:24:33.349067 140205107017536 deprecation.py:506] From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (fro

I0701 23:24:36.892337 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_7/output/dense/bias:0, shape = (768,)
I0701 23:24:36.892462 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_7/output/LayerNorm/beta:0, shape = (768,)
I0701 23:24:36.892585 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_7/output/LayerNorm/gamma:0, shape = (768,)
I0701 23:24:36.892710 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_8/attention/self/query/kernel:0, shape = (768, 768)
I0701 23:24:36.892858 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_8/attention/self/query/bias:0, shape = (768,)
I0701 23:24:36.892999 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_8/attention/self/key/kernel:0, shape = (768, 768)
I0701 23:24:36.893168 140205107017536 run_pretraining.py:173]   name = bert/encoder/layer_8/attention/self/key/bias:0, shape = (768,)
I0701 23:24:36.893296 140205107017536 run_pretraining.p

W0701 23:24:37.346451 140205107017536 module_wrapper.py:139] From run_pretraining.py:198: The name tf.metrics.accuracy is deprecated. Please use tf.compat.v1.metrics.accuracy instead.

W0701 23:24:37.364413 140205107017536 module_wrapper.py:139] From run_pretraining.py:202: The name tf.metrics.mean is deprecated. Please use tf.compat.v1.metrics.mean instead.

I0701 23:24:37.425478 140205107017536 estimator.py:1150] Done calling model_fn.
I0701 23:24:37.446460 140205107017536 evaluation.py:255] Starting evaluation at 2020-07-01T23:24:37Z
I0701 23:24:37.446757 140205107017536 tpu_estimator.py:506] TPU job name worker
W0701 23:24:37.585012 140205107017536 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:1475: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0701 23:24:38.012066 1402051070175

I0701 23:25:02.038308 140205107017536 estimator.py:2109] Saving 'checkpoint_path' summary for global step 20: gs://tf-lang-model_danzter/model/model.ckpt-20
2020-07-01 23:25:02.039350: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Cancelled: GCE check skipped due to presence of $NO_GCE_CHECK environment variable.".
2020-07-01 23:25:02.108354: W tensorflow/core/platform/cloud/google_auth_provider.cc:178] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Cancelled: GCE check skipped due to presence of $NO_GCE_CHECK environment variable.".
2020-07-01 23:25:02.604110: W tensorflow/core